Test a bunch of notebooks. Bioinfo notebooks are at ~/repos/bioinfo/test.ipynb

In [1]:
from k1lib.imports import *
from IPython.display import clear_output

In [2]:
def uncommentTestCells(cell):
    if [cell] | nb.pretty(whitelist=["test"]) | shape() | item() > 0:
        cell["source"][1:] = [line.lstrip("# ") for line in cell["source"][1:]]
    return cell

In [3]:
def loadNb(fileName):
    return nb.cells(fileName) | nb.pretty(blacklist=["notest"]) | filt(op()["cell_type"] == "code") | apply(uncommentTestCells)
def execute(fileName:str):
    print(fileName)
    try: loadNb(fileName) | nb.execute(fileName)
    except pickle.PicklingError: # for some reason, applyMp sort of has to pass in the current globals(), else it throws pickling errors
        clear_output(); loadNb(fileName) | nb.execute(fileName, globals())
    clear_output(); applyMp.clearPools()

In [6]:
base = "/home/kelvin/repos/labs/k1lib/k1lib"
tests = []
for i in range(5): tests += glob.glob(f"{base}{'/*' * i}.ipynb")
tests.remove(f"{base}/test.ipynb")
tests.remove(f"{base}/cli/trash.ipynb")
for i, test in enumerate(tests):
    print(i); execute(test)

In [4]:
tut = "/home/kelvin/repos/labs/k1lib/docs/tutorials"
ls(tut) | grep(".ipynb$") | headOut()

/home/kelvin/repos/labs/k1lib/docs/tutorials/basics.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/mp.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/schedule.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/mo.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/osic.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/cli.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/selector.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/trace.ipynb
/home/kelvin/repos/labs/k1lib/docs/tutorials/covid.ipynb


In [5]:
execute(f"{tut}/selector.ipynb")
execute(f"{tut}/trace.ipynb")
execute(f"{tut}/covid.ipynb")
execute(f"{tut}/cli.ipynb")
execute(f"{tut}/schedule.ipynb")
execute(f"{tut}/mo.ipynb")
execute(f"{tut}/osic.ipynb")
execute(f"{tut}/mp.ipynb")
execute(f"{tut}/basics.ipynb")

In [6]:
execute("/home/kelvin/repos/labs/projects/char rnn/1) init/1) init.ipynb")

In [7]:
execute("/home/kelvin/repos/labs/projects/function/0) integrity/main.ipynb")

In [8]:
execute("/home/kelvin/repos/labs/projects/mnist/setup.ipynb")

In [4]:
os.chdir("~/repos/labs/mlexps")
niceFolders = lambda: filt(os.path.isdir) | ~filt("ipynb" in op())

In [5]:
i = 0
for folder in ls(".") | niceFolders():
    with settings.context(wd=folder):
        for f in ls(".") | niceFolders():
            i += 1; os.chdir(f); ipy = f + ".ipynb"; py = ipy.replace(".ipynb", ".py")
            selection = os.getcwd()
            loadNb(ipy) | apply(op()["source"]) | (join("\n")).all() | file(py)
            None | cmd(f"python {py}") | headOut(None)
            os.chdir("..")

<IPython.core.display.HTML object>
Epoch cancelled: Batch 1 reached.
Epoch cancelled: Batch 1 reached.
Epoch cancelled: Batch 1 reached.
Epoch cancelled: Batch 1 reached.
applyMp encounter errors. Terminating pool now

Error encountered:

Traceback (most recent call last):
  File "./3-vanilla.py", line 48, in <module>
    newL().run(1)
  File "/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/k1lib-0.14-py3.8.egg/k1lib/_learner.py", line 195, in run
  File "/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/k1lib-0.14-py3.8.egg/k1lib/_learner.py", line 166, in _run1Epoch
  File "/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/k1lib-0.14-py3.8.egg/k1lib/_learner.py", line 155, in __iter__
  File "/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/k1lib-0.14-py3.8.egg/k1lib/cli/modifier.py", line 424, in __iter__
  File "/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/k1lib-0.14-py3.8.egg/k1lib/cli/structural.py", line 186, in

In [6]:
k1lib.beep()